In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kaust\\Desktop\\Text-Summarizer-Project\\research'

In [ ]:
#change the dir to root folder

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\kaust\\Desktop\\Text-Summarizer-Project'

In [5]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [7]:
#configuration manager in src config is updated
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,     #definig constants
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )

          return data_ingestion_config


        
    

In [8]:
# updating the components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size




In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns none
        """      
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




In [10]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-02 03:10:09,289: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-09-02 03:10:09,319: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-09-02 03:10:09,325: INFO: common: created directory at: artifacts]
[2024-09-02 03:10:09,332: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-02 03:10:23,592: INFO: 3354448272: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2AD8:554A8:2C671E:39A9B4:66D58F05
Accept-Ranges: bytes
Date: Mon, 02 Sep 2024 10:10:14 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100027-HYD
X-Cache: M

EOFError: 

In [ ]:
#create the pipeline